In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [25]:
troll_1 = pd.read_csv('data/IRAhandle_tweets_1.csv')
troll_2 = pd.read_csv('data/IRAhandle_tweets_2.csv')
troll_3 = pd.read_csv('data/IRAhandle_tweets_3.csv')
troll_4 = pd.read_csv('data/IRAhandle_tweets_4.csv')
troll_5 = pd.read_csv('data/IRAhandle_tweets_5.csv')
troll_6 = pd.read_csv('data/IRAhandle_tweets_6.csv')
troll_7 = pd.read_csv('data/IRAhandle_tweets_7.csv')
troll_8 = pd.read_csv('data/IRAhandle_tweets_8.csv')
troll_9 = pd.read_csv('data/IRAhandle_tweets_9.csv')

frames = [troll_1, troll_2, troll_3, troll_4, troll_5, troll_6, troll_7, troll_8, troll_8]

troll_df = pd.concat(frames)

In [29]:
troll_df.dropna(how='all')
troll_df.drop(troll_df[troll_df['language'] != 'English'].index, inplace = True)
troll_df = troll_df.sample(frac=0.66, replace=False, random_state=1)
troll_df.shape

(1588998, 15)

In [30]:
troll_df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
64460,1.690491e+09,RAYEWHALEN,"After 6 seasons but no movie (yet), is this th...",United States,English,6/1/2015 20:35,6/1/2015 20:35,112,147,616,RETWEET,Right,0,1,RightTroll
344647,3.241517e+09,FREEDISCUSS,IF CHARACTER HAS ANYTHING TO DO WITH PRESIDENT...,United States,English,6/14/2015 13:59,6/14/2015 13:59,3,18,1227,NaN,Right,0,0,RightTroll
188921,2.601236e+09,TODAYPITTSBURGH,State Police Investigating Attempted Luring in...,United States,English,9/23/2015 8:09,9/23/2015 8:09,3274,13160,15860,NaN,local,0,0,NewsFeed
190369,1.676481e+09,EMILEEWAREN,#BLMKidnapping #BrittanyHerring Just so you kn...,United States,English,1/5/2017 2:41,1/5/2017 2:41,1872,2094,3782,RETWEET,Right,0,1,RightTroll
52072,4.224730e+09,TEN_GOP,Danny Trejo: You can be proud of your Mexican ...,United States,English,9/15/2016 17:53,9/15/2016 17:53,25454,30353,3817,NaN,Right,0,0,RightTroll


In [32]:
labels_to_drop = ['external_author_id', 'author', 'region', 'language', 'publish_date', 'harvested_date', 'following', 'followers', 'updates', 'post_type', 'account_type', 'new_june_2018', 'retweet', 'account_category']
troll_df = troll_df.drop(labels_to_drop, axis=1)


,content
64460,"After 6 seasons but no movie (yet), is this th..."
344647,IF CHARACTER HAS ANYTHING TO DO WITH PRESIDENT...
188921,State Police Investigating Attempted Luring in...
190369,#BLMKidnapping #BrittanyHerring Just so you kn...
52072,Danny Trejo: You can be proud of your Mexican ...


In [33]:
troll_df['class'] = 1

In [35]:
troll_df.head()

,content,class
64460,"After 6 seasons but no movie (yet), is this th...",1
344647,IF CHARACTER HAS ANYTHING TO DO WITH PRESIDENT...,1
188921,State Police Investigating Attempted Luring in...,1
190369,#BLMKidnapping #BrittanyHerring Just so you kn...,1
52072,Danny Trejo: You can be proud of your Mexican ...,1


In [45]:
real_tweet_df = pd.read_csv('data/training.1600000.processed.noemoticon.csv', header=None, usecols=[5], encoding='ISO-8859-1')

In [51]:
real_tweet_df=real_tweet_df.rename(columns={5: "content"})
real_tweet_df.head()

,content
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,is upset that he can't update his Facebook by ...
2,@Kenichan I dived many times for the ball. Man...
3,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all...."


In [52]:
real_tweet_df['class'] = 0

In [53]:
real_tweet_df.head()

,content,class
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [54]:
frames_real_fake = [real_tweet_df, troll_df]
tweets_df = pd.concat(frames_real_fake)
tweets_df.head()

,content,class
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [55]:
tweets_df.tail()

,content,class
242847,Atlanta Police searching for man wanted for ra...,1
251888,We REALLY fuckin' listen 2 your music submissi...,1
225033,Thank you! Love to travel with readers. You're...,1
47724,"He never calls back, guess im annoying",1
321480,So your racism has been effectively proven @em...,1


In [56]:
tweets_df.shape

(3188998, 2)

In [57]:
tweets_df.to_csv('tweets.csv', index=False)